In [1]:
%%writefile contents_lab.json


{
    "components": [
        {
            "name": "resistor",
            "values": [
                {
                    "resistance": "10 ohms",
                    "availability": "Yes, this resistor is available in the lab.",
                    "count":20
                },
                {
                    "resistance": "100 ohms",
                    "availability": "Yes, this resistor is available in the lab."
                }
            ]
        },
        {
            "name": "inductor",
            "values": [
                {
                    "inductance": "100 microhenries",
                    "availability":"Yes,this inductor is available in the lab.",
                    "count":40
                },
                {
                    "inductance": "1 millihenry",
                    "availability": "No, this inductor is not available in the lab."
                }
            ]
        },
        {
            "name": "capacitor",
            "values": [
                {
                    "capacitance": "10 microfarads",
                    "availability": "Yes, this capacitor is available in the lab.",
                    "count":70
                },
                {
                    "capacitance": "100 microfarads",
                    "availability": "Yes, this capacitor is available in the lab."
                }
            ]
        },
        {
            "name": "transistor",
            "values": [
                {
                    "type": "npn",
                    "model": "2N2222",
                    "availability": "Yes, this transistor is available in the lab."
                },
                {
                    "type": "pnp",
                    "model": "2N2907",
                    "availability": "Yes, this transistor is available in the lab."
                }
            ]
        },
        {
            "name": "mosfet",
            "values": [
                {
                    "type": "n channel",
                    "model": "IRF540",
                    "availability": "Yes, this MOSFET is available in the lab."
                },
                {
                    "type": "p channel",
                    "model": "IRF9540",
                    "availability": "No, this MOSFET is not available in the lab."
                }
            ]
        },
        {
            "name": "ic",
            "values": [
                {
                    "type": "operational amplifier",
                    "model": "LM358",
                    "availability": "Yes, this IC is available in the lab."
                },
                {
                    "type": "555 timer",
                    "model": "LM555",
                    "availability": "Yes, this IC is available in the lab."
                }
            ]
        }
    ]
}


Writing contents_lab.json


In [2]:
import json
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import pyttsx3
import speech_recognition as sr

#initialize text to speech
engine=pyttsx3.init()

#initialize speech recognizer
recognizer=sr.Recognizer()

#load components data from json
def load_components():
    with open('contents_lab.json','r') as file:
        data=json.load(file)
    return data['components']
components=load_components()

#func to preprocess the user's question
def preprocess_question(question):
    stop_words=set(stopwords.words('english'))
    tokens=word_tokenize(question.lower())
    filtered_tokens=[word for word in tokens if word.isalnm() and word not in stop_words]
    return'_'.join(filtered_tokens)

#function to provide audio feedback
def provide_audio_feedback(text):
    engine.say(text)
    engine.runAndWait()

#func to check availability of a specific component
def check_availability(component_name,attribute_name,attribute_value):
    count=0
    selected_component=next((comp for comp in components if comp['name'].lower()==component_name.lower()),None)
    if not selected_component:
        response="Component not found.Please enter a valid component name."
        return response
    
    for value in selected_component['values']:
        if value.get(attribute_name)==attribute_value:
            count+=1

    if count>0:
        return f"There are{count}{attribute_value}{component_name} of {attribute_value} found in the lab."
    else:
        return f"No {component_name} with {attribute_name} of {attribute_value} found in the lab."
    
#func for user input from microphone
def get_speech_input(prompt):
    with sr.Microphone() as source:
        print(prompt)
        provide_audio_feedback(prompt)
        audio=recognizer.listen(source)
        try:
            text=recognizer.recognize_google(audio)
            print(f"You Said:{text}")
            return text
        except sr.UnknownValueError:
            print("Sorry, I did not understand that.")
            provide_audio_feedback("Sorry, I did not understand that.")
            return None
        except sr.RequestError:
            print("Sorry, there was an issue with the speech recognition service.")
            provide_audio_feedback("Sorry, there was an issue with the speech recognition service.")
            return None

In [3]:

# Function for checking interaction
def simulate_component_check():
    while True:
        component_name = get_speech_input("Enter the component name or say 'exit' to quit:")
        if component_name and component_name.lower() in ['exit', 'quit', 'stop']:
            provide_audio_feedback("Ending the component check.")
            print("Ending the component check.")
            break

        attribute_name = get_speech_input("Enter the attribute you want to check:")
        if not attribute_name:
            continue

        attribute_value = get_speech_input(f"Enter the value for {attribute_name}:")
        if not attribute_value:
            continue

        response = check_availability(component_name, attribute_name, attribute_value)
        print(response)
        provide_audio_feedback(response)


In [ ]:
# Run the component check simulation
simulate_component_check()